In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import os
import random
from torch.autograd import Variable
import copy
from torch import nn, optim
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import csv
import time


In [2]:
def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
fix_seed(SEED)

In [3]:
class Argments():
  def __init__(self):
    self.batch_size = 20
    self.test_batch = 1000
    #self.global_epochs = 300
    self.local_epochs = 200
    self.lr = None
    self.momentum = 0.9
    self.weight_decay = 10**-4.0
    self.clip = 20.0
    self.partience = 200
    self.worker_num = 20
    #self.sample_num = 20
    self.device = device = torch.device('cuda:0'if torch.cuda.is_available() else'cpu')
    self.criterion = nn.CrossEntropyLoss()

args = Argments()

In [4]:
# Tuned value
lr = 1

In [6]:
lr_list = []
lr_list.append(10**-3.0)
lr_list.append(10**-2.5)
lr_list.append(10**-2.0)
lr_list.append(10**-1.5)
lr_list.append(10**-1.0)
lr_list.append(10**-0.5)
lr_list.append(10**0.0)
lr_list.append(10**0.5)

args.lr = lr_list[lr]

In [9]:
class LocalDataset(torch.utils.data.Dataset):
  def __init__(self,dataset,worker_id):
    self.data = []
    self.target = []
    self.id = worker_id
    for data in dataset.take(len(dataset)):
      self.data.append(torch.tensor([data['pixels'].numpy()]))
      self.target.append(torch.tensor(data['label'].numpy().astype(np.int64)))

  def __getitem__(self, index):
    return self.data[index],self.target[index]

  def __len__(self):
    return len(self.data)

In [10]:
with open('../data/federated_trainset_femnist.pickle', 'rb') as f:
    all_federated_trainset = pickle.load(f)
with open('../data/federated_testset_femnist.pickle', 'rb') as f:
    all_federated_testset = pickle.load(f)
all_worker_num = len(all_federated_trainset)

In [11]:
worker_id_list = random.sample(range(all_worker_num),args.worker_num)
print(worker_id_list)
federated_trainset = []
federated_testset = []
for i in worker_id_list:
    federated_trainset.append(all_federated_trainset[i])
    federated_testset.append(all_federated_testset[i])

[2619, 456, 102, 3037, 1126, 1003, 914, 571, 3016, 419, 2771, 3033, 2233, 356, 2418, 1728, 130, 122, 383, 895]


In [12]:
federated_valset = [None]*args.worker_num
for i in range(args.worker_num):
  n_samples = len(federated_trainset[i])
  if n_samples==1:
    federated_valset[i] = copy.deepcopy(federated_trainset[i])
  else:
    train_size = int(len(federated_trainset[i]) * 0.7) 
    val_size = n_samples - train_size 
    federated_trainset[i],federated_valset[i] = torch.utils.data.random_split(federated_trainset[i], [train_size, val_size])

In [14]:
class CNN1(torch.nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(5408, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

class CNN2(torch.nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(9216, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

class CNN3(torch.nn.Module):
    def __init__(self):
        super(CNN3, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.conv2d_3 = torch.nn.Conv2d(64, 128, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(15488, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.conv2d_3(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x
    
class CNN4(torch.nn.Module):
    def __init__(self):
        super(CNN4, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.conv2d_3 = torch.nn.Conv2d(64, 128, kernel_size=3)
        self.conv2d_4 = torch.nn.Conv2d(128, 256, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(25600, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.conv2d_3(x)
        x = self.conv2d_4(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

In [15]:
class Server():
  def __init__(self):
    self.model = CNN2()

  def create_worker(self,federated_trainset,federated_valset,federated_testset):
    workers = []
    for i in range(args.worker_num):
      workers.append(Worker(federated_trainset[i],federated_valset[i],federated_testset[i]))
    return workers

  def sample_worker(self,workers):
    sample_worker = []
    sample_worker_num = random.sample(range(args.worker_num),args.sample_num)
    for i in sample_worker_num:
      sample_worker.append(workers[i])
    return sample_worker


  def send_model(self,workers):
    nums = 0
    for worker in workers:
      nums += worker.train_data_num

    for worker in workers:
      worker.aggregation_weight = 1.0*worker.train_data_num/nums
      worker.model = copy.deepcopy(self.model)
      worker.model = worker.model.to(args.device)

  def aggregate_model(self,workers):   
    new_params = OrderedDict()
    for i,worker in enumerate(workers):
      worker_state = worker.model.state_dict()
      for key in worker_state.keys():
        if i==0:
          new_params[key] = worker_state[key]*worker.aggregation_weight
        else:
          new_params[key] += worker_state[key]*worker.aggregation_weight
      worker.model = worker.model.to('cpu')
      del worker.model
    self.model.load_state_dict(new_params)

In [16]:
class Worker():
  def __init__(self,trainset,valset,testset):
    self.trainloader = torch.utils.data.DataLoader(trainset,batch_size=args.batch_size,shuffle=True,num_workers=2)
    self.valloader = torch.utils.data.DataLoader(valset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.testloader = torch.utils.data.DataLoader(testset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.model = None
    self.train_data_num = len(trainset)
    self.test_data_num = len(testset)
    self.aggregation_weight = None

  def local_train(self):
    acc_valid,loss_valid = train(self.model,args.criterion,self.trainloader,self.valloader,args.local_epochs,partience=args.partience,early_stop=True)
    return acc_valid[-1],loss_valid[-1]

In [17]:
def train(model,criterion,trainloader,valloader,epochs,partience=0,early_stop=False):
  if early_stop:
    early_stopping = Early_Stopping(partience)

  acc_valid = []
  loss_valid = []
  optimizer = optim.SGD(model.parameters(),lr=args.lr,momentum=args.momentum,weight_decay=args.weight_decay)
  for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    count = 0
    model.train()
    for (data,labels) in trainloader:
      data,labels = Variable(data),Variable(labels)
      data,labels = data.to(args.device),labels.to(args.device)
      optimizer.zero_grad()
      outputs = model(data)
      loss = criterion(outputs,labels)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
      optimizer.step()
      
    model.eval()
    for (data,labels) in valloader:
      count += len(labels)
      data,labels = data.to(args.device),labels.to(args.device)
      outputs = model(data)
      loss = criterion(outputs,labels)
      running_loss += loss.item()
      predicted = torch.argmax(outputs,dim=1)
      correct += (predicted==labels).sum().item()
      
    print('Epoch:{}  accuracy:{}  loss:{}'.format(epoch+1,100.0*correct/count,running_loss/len(valloader)))
    acc_valid.append(100.0*correct/count)
    loss_valid.append(running_loss/len(valloader))
    if early_stop:
      if early_stopping.validate(running_loss):
        print('Early Stop')
        return acc_valid,loss_valid

  return acc_valid,loss_valid

In [18]:
def test(model,criterion,testloader):
  model.eval()
  running_loss = 0.0
  correct = 0
  count = 0
  for (data,labels) in testloader:
    data,labels = data.to(args.device),labels.to(args.device)
    outputs = model(data)
    running_loss += criterion(outputs,labels).item()
    predicted = torch.argmax(outputs,dim=1)
    correct += (predicted==labels).sum().item()
    count += len(labels)

  accuracy = 100.0*correct/count
  loss = running_loss/len(testloader)


  return accuracy,loss

In [19]:
class Early_Stopping():
  def __init__(self,partience):
    self.step = 0
    self.loss = float('inf')
    self.partience = partience

  def validate(self,loss):
    if self.loss<loss:
      self.step += 1
      if self.step>self.partience:
        return True
    else:
      self.step = 0
      self.loss = loss

    return False

In [20]:
server = Server()
workers = server.create_worker(federated_trainset,federated_valset,federated_testset)
acc_valid = []
loss_valid = []

early_stopping = Early_Stopping(args.partience)

start = time.time()

for i,worker in enumerate(workers):
    print('Worker{} start'.format(i+1))
    worker.model = CNN2()
    worker.model = worker.model.to(args.device)
    acc_tmp,loss_tmp = worker.local_train()
    acc_valid.append(acc_tmp)
    loss_valid.append(loss_tmp)
    worker.model = worker.model.to('cpu')
    
end = time.time()

Worker1 start
Epoch:1  accuracy:4.651162790697675  loss:4.009448528289795
Epoch:2  accuracy:4.651162790697675  loss:3.846885919570923
Epoch:3  accuracy:4.651162790697675  loss:3.729663610458374
Epoch:4  accuracy:4.651162790697675  loss:3.6657111644744873
Epoch:5  accuracy:4.651162790697675  loss:3.6388397216796875
Epoch:6  accuracy:11.627906976744185  loss:3.6087863445281982
Epoch:7  accuracy:4.651162790697675  loss:3.5904483795166016
Epoch:8  accuracy:4.651162790697675  loss:3.5911014080047607
Epoch:9  accuracy:4.651162790697675  loss:3.590278148651123
Epoch:10  accuracy:4.651162790697675  loss:3.5832321643829346
Epoch:11  accuracy:4.651162790697675  loss:3.5808112621307373
Epoch:12  accuracy:6.976744186046512  loss:3.587150812149048
Epoch:13  accuracy:6.976744186046512  loss:3.591174364089966
Epoch:14  accuracy:4.651162790697675  loss:3.5847842693328857
Epoch:15  accuracy:4.651162790697675  loss:3.6084442138671875
Epoch:16  accuracy:4.651162790697675  loss:3.6088199615478516
Epoch:17

Epoch:134  accuracy:62.7906976744186  loss:2.8030781745910645
Epoch:135  accuracy:62.7906976744186  loss:2.8389499187469482
Epoch:136  accuracy:65.11627906976744  loss:2.823789596557617
Epoch:137  accuracy:69.76744186046511  loss:2.785820484161377
Epoch:138  accuracy:62.7906976744186  loss:2.827394962310791
Epoch:139  accuracy:62.7906976744186  loss:2.816401243209839
Epoch:140  accuracy:65.11627906976744  loss:2.8426671028137207
Epoch:141  accuracy:67.44186046511628  loss:2.892025947570801
Epoch:142  accuracy:65.11627906976744  loss:3.0027780532836914
Epoch:143  accuracy:65.11627906976744  loss:2.871861219406128
Epoch:144  accuracy:62.7906976744186  loss:2.791124105453491
Epoch:145  accuracy:65.11627906976744  loss:2.779473066329956
Epoch:146  accuracy:67.44186046511628  loss:2.8496124744415283
Epoch:147  accuracy:67.44186046511628  loss:2.946240186691284
Epoch:148  accuracy:65.11627906976744  loss:2.988650321960449
Epoch:149  accuracy:65.11627906976744  loss:3.0071818828582764
Epoch:1

Epoch:66  accuracy:51.06382978723404  loss:2.3894081115722656
Epoch:67  accuracy:45.744680851063826  loss:2.371117115020752
Epoch:68  accuracy:48.93617021276596  loss:2.295610189437866
Epoch:69  accuracy:46.808510638297875  loss:2.363753080368042
Epoch:70  accuracy:48.93617021276596  loss:2.2826085090637207
Epoch:71  accuracy:45.744680851063826  loss:2.2666821479797363
Epoch:72  accuracy:51.06382978723404  loss:2.186246395111084
Epoch:73  accuracy:45.744680851063826  loss:2.2665293216705322
Epoch:74  accuracy:50.0  loss:2.203979253768921
Epoch:75  accuracy:53.191489361702125  loss:2.164306402206421
Epoch:76  accuracy:54.255319148936174  loss:2.1138863563537598
Epoch:77  accuracy:54.255319148936174  loss:2.114805221557617
Epoch:78  accuracy:57.4468085106383  loss:2.057065963745117
Epoch:79  accuracy:58.51063829787234  loss:2.0298008918762207
Epoch:80  accuracy:57.4468085106383  loss:2.006199836730957
Epoch:81  accuracy:57.4468085106383  loss:2.0263044834136963
Epoch:82  accuracy:60.6382

Epoch:198  accuracy:76.59574468085107  loss:1.9290878772735596
Epoch:199  accuracy:76.59574468085107  loss:1.920681357383728
Epoch:200  accuracy:76.59574468085107  loss:1.962894320487976
Worker3 start
Epoch:1  accuracy:2.857142857142857  loss:4.101900577545166
Epoch:2  accuracy:10.476190476190476  loss:4.020320415496826
Epoch:3  accuracy:2.857142857142857  loss:3.9648571014404297
Epoch:4  accuracy:2.857142857142857  loss:3.883856773376465
Epoch:5  accuracy:2.857142857142857  loss:3.823458433151245
Epoch:6  accuracy:10.476190476190476  loss:3.7669501304626465
Epoch:7  accuracy:2.857142857142857  loss:3.7465715408325195
Epoch:8  accuracy:2.857142857142857  loss:3.7372207641601562
Epoch:9  accuracy:2.857142857142857  loss:3.7434985637664795
Epoch:10  accuracy:2.857142857142857  loss:3.728687286376953
Epoch:11  accuracy:2.857142857142857  loss:3.7168288230895996
Epoch:12  accuracy:10.476190476190476  loss:3.690178394317627
Epoch:13  accuracy:6.666666666666667  loss:3.6809704303741455
Epoch

Epoch:131  accuracy:65.71428571428571  loss:1.9099539518356323
Epoch:132  accuracy:66.66666666666667  loss:1.9457180500030518
Epoch:133  accuracy:66.66666666666667  loss:1.8472093343734741
Epoch:134  accuracy:67.61904761904762  loss:1.9019800424575806
Epoch:135  accuracy:65.71428571428571  loss:1.8325369358062744
Epoch:136  accuracy:62.857142857142854  loss:1.8867872953414917
Epoch:137  accuracy:65.71428571428571  loss:1.8294905424118042
Epoch:138  accuracy:63.80952380952381  loss:1.883902668952942
Epoch:139  accuracy:64.76190476190476  loss:1.9081872701644897
Epoch:140  accuracy:65.71428571428571  loss:1.893852710723877
Epoch:141  accuracy:64.76190476190476  loss:1.9315768480300903
Epoch:142  accuracy:64.76190476190476  loss:1.9413909912109375
Epoch:143  accuracy:63.80952380952381  loss:1.9616174697875977
Epoch:144  accuracy:64.76190476190476  loss:1.92447829246521
Epoch:145  accuracy:64.76190476190476  loss:1.9215428829193115
Epoch:146  accuracy:62.857142857142854  loss:1.96956348419

...

Epoch:174  accuracy:64.77272727272727  loss:1.856449842453003
Epoch:175  accuracy:65.9090909090909  loss:1.814937710762024
Epoch:176  accuracy:67.04545454545455  loss:1.8117963075637817
Epoch:177  accuracy:69.31818181818181  loss:1.8307926654815674
Epoch:178  accuracy:67.04545454545455  loss:1.8609081506729126
Epoch:179  accuracy:65.9090909090909  loss:1.802201509475708
Epoch:180  accuracy:67.04545454545455  loss:1.7820239067077637
Epoch:181  accuracy:67.04545454545455  loss:1.7844176292419434
Epoch:182  accuracy:67.04545454545455  loss:1.8171768188476562
Epoch:183  accuracy:67.04545454545455  loss:1.8913391828536987
Epoch:184  accuracy:68.18181818181819  loss:1.8518586158752441
Epoch:185  accuracy:70.45454545454545  loss:1.745627999305725
Epoch:186  accuracy:68.18181818181819  loss:1.7358342409133911
Epoch:187  accuracy:65.9090909090909  loss:1.8343526124954224
Epoch:188  accuracy:65.9090909090909  loss:1.8469643592834473
Epoch:189  accuracy:69.31818181818181  loss:1.8355919122695923


Epoch:108  accuracy:53.65853658536585  loss:2.2129836082458496
Epoch:109  accuracy:53.65853658536585  loss:2.1500277519226074
Epoch:110  accuracy:50.0  loss:2.1785387992858887
Epoch:111  accuracy:53.65853658536585  loss:2.149341344833374
Epoch:112  accuracy:51.21951219512195  loss:2.1268815994262695
Epoch:113  accuracy:52.4390243902439  loss:2.187537908554077
Epoch:114  accuracy:54.8780487804878  loss:2.1515698432922363
Epoch:115  accuracy:53.65853658536585  loss:2.2006304264068604
Epoch:116  accuracy:53.65853658536585  loss:2.1692733764648438
Epoch:117  accuracy:53.65853658536585  loss:2.157712697982788
Epoch:118  accuracy:54.8780487804878  loss:2.2425649166107178
Epoch:119  accuracy:56.09756097560975  loss:2.1920626163482666
Epoch:120  accuracy:53.65853658536585  loss:2.2755563259124756
Epoch:121  accuracy:56.09756097560975  loss:2.2075741291046143
Epoch:122  accuracy:53.65853658536585  loss:2.227597236633301
Epoch:123  accuracy:53.65853658536585  loss:2.2442691326141357
Epoch:124  a

Epoch:41  accuracy:16.666666666666668  loss:3.6343579292297363
Epoch:42  accuracy:20.0  loss:3.617316722869873
Epoch:43  accuracy:13.333333333333334  loss:3.60634183883667
Epoch:44  accuracy:12.222222222222221  loss:3.5984046459198
Epoch:45  accuracy:12.222222222222221  loss:3.5866336822509766
Epoch:46  accuracy:22.22222222222222  loss:3.557870864868164
Epoch:47  accuracy:13.333333333333334  loss:3.5494384765625
Epoch:48  accuracy:12.222222222222221  loss:3.5517425537109375
Epoch:49  accuracy:13.333333333333334  loss:3.5386664867401123
Epoch:50  accuracy:14.444444444444445  loss:3.522500991821289
Epoch:51  accuracy:13.333333333333334  loss:3.5176334381103516
Epoch:52  accuracy:13.333333333333334  loss:3.483339548110962
Epoch:53  accuracy:18.88888888888889  loss:3.469691276550293
Epoch:54  accuracy:21.11111111111111  loss:3.4428741931915283
Epoch:55  accuracy:20.0  loss:3.4150493144989014
Epoch:56  accuracy:23.333333333333332  loss:3.392490863800049
Epoch:57  accuracy:21.11111111111111 

Epoch:176  accuracy:74.44444444444444  loss:1.6164337396621704
Epoch:177  accuracy:76.66666666666667  loss:1.5746440887451172
Epoch:178  accuracy:77.77777777777777  loss:1.5677242279052734
Epoch:179  accuracy:75.55555555555556  loss:1.5739130973815918
Epoch:180  accuracy:74.44444444444444  loss:1.588486909866333
Epoch:181  accuracy:76.66666666666667  loss:1.5758265256881714
Epoch:182  accuracy:74.44444444444444  loss:1.6222708225250244
Epoch:183  accuracy:76.66666666666667  loss:1.5946706533432007
Epoch:184  accuracy:76.66666666666667  loss:1.5499825477600098
Epoch:185  accuracy:74.44444444444444  loss:1.5880439281463623
Epoch:186  accuracy:74.44444444444444  loss:1.5908844470977783
Epoch:187  accuracy:77.77777777777777  loss:1.6032334566116333
Epoch:188  accuracy:77.77777777777777  loss:1.652908444404602
Epoch:189  accuracy:77.77777777777777  loss:1.6285834312438965
Epoch:190  accuracy:74.44444444444444  loss:1.6638987064361572
Epoch:191  accuracy:74.44444444444444  loss:1.67008078098

Epoch:177  accuracy:42.22222222222222  loss:5.662705421447754
Epoch:178  accuracy:40.0  loss:5.622132778167725
Epoch:179  accuracy:42.22222222222222  loss:5.648148536682129
Epoch:180  accuracy:42.22222222222222  loss:5.711604118347168
Epoch:181  accuracy:42.22222222222222  loss:5.643690586090088
Epoch:182  accuracy:42.22222222222222  loss:5.456447601318359
Epoch:183  accuracy:42.22222222222222  loss:5.518252372741699
Epoch:184  accuracy:42.22222222222222  loss:5.737865447998047
Epoch:185  accuracy:44.44444444444444  loss:6.025311470031738
Epoch:186  accuracy:42.22222222222222  loss:5.786643981933594
Epoch:187  accuracy:40.0  loss:5.514491558074951
Epoch:188  accuracy:40.0  loss:5.684360980987549
Epoch:189  accuracy:40.0  loss:6.14538049697876
Epoch:190  accuracy:40.0  loss:6.245247840881348
Epoch:191  accuracy:40.0  loss:6.113632678985596
Epoch:192  accuracy:40.0  loss:6.107842445373535
Epoch:193  accuracy:40.0  loss:6.145058631896973
Epoch:194  accuracy:42.22222222222222  loss:6.03277

In [22]:
acc_test = []
loss_test = []

start = time.time()

for i,worker in enumerate(workers):
    worker.model = worker.model.to(args.device)
    acc_tmp,loss_tmp = test(worker.model,args.criterion,worker.testloader)
    acc_test.append(acc_tmp)
    loss_test.append(loss_tmp)
    print('Worker{} accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))
    worker.model = worker.model.to('cpu')
    
end = time.time()

Worker1 accuracy:64.70588235294117  loss:4.076274871826172
Worker2 accuracy:91.42857142857143  loss:0.7707714438438416
Worker3 accuracy:77.5  loss:1.4779036045074463
Worker4 accuracy:55.55555555555556  loss:4.387485027313232
Worker5 accuracy:76.47058823529412  loss:1.1927577257156372
Worker6 accuracy:61.904761904761905  loss:2.379671812057495
Worker7 accuracy:68.42105263157895  loss:2.490126848220825
Worker8 accuracy:59.375  loss:2.559020519256592
Worker9 accuracy:52.63157894736842  loss:3.5772740840911865
Worker10 accuracy:87.8048780487805  loss:0.5430195927619934
Worker11 accuracy:55.55555555555556  loss:4.74709939956665
Worker12 accuracy:68.42105263157895  loss:3.9164011478424072
Worker13 accuracy:52.63157894736842  loss:4.088058948516846
Worker14 accuracy:40.0  loss:3.586329221725464
Worker15 accuracy:73.6842105263158  loss:3.1282455921173096
Worker16 accuracy:52.94117647058823  loss:3.898005247116089
Worker17 accuracy:69.6969696969697  loss:1.769512414932251
Worker18 accuracy:67.7

In [24]:
acc_valid_avg = sum(acc_valid)/len(acc_valid)
loss_valid_avg = sum(loss_valid)/len(loss_valid)
print('Validation  loss:{}  accuracy:{}'.format(loss_valid_avg,acc_valid_avg))

acc_test_avg = sum(acc_test)/len(acc_test)
loss_test_avg = sum(loss_test)/len(loss_test)
print('Test  loss:{}  accuracy:{}'.format(loss_test_avg,acc_test_avg))

Validation  loss:2.8830289959907534  accuracy:66.24876712753404
Test  loss:2.8013279855251314  accuracy:65.1470468326197
